# SC32x 
## 자연어처리 (Natural Language Processing)

# Part 1 : 개념 요약

> 다음의 키워드에 대해서 **한 줄**로 간단하게 요약해주세요. (세션 노트를 참고하여도 좋습니다.)<br/>
> **Tip : 아래 문제를 먼저 수행한 후 모델 학습 등 시간이 오래 걸리는 셀이 실행되는 동안 아래 내용을 작성하면 시간을 절약할 수 있습니다.**

**N321**
- Stopwords(불용어)
- Stemming과 Lemmatization
- Bag-of-Words
- TF-IDF

***이곳에 답안을 입력해주세요***

Stopwords : 텍스트를 토큰화할 때 의미가 없는 단어로 불용어라고 불림, 이는 분석할 때 해당 단어를 제외하고 계산하기 위해 분류를 하는 것이 중요함.

Stemming과 Lemmatization : 어간 추출과 표제어 추출로 토큰화된 단어들에서 Stemming은 어간추출로 단어의 뒷 부분이 제거된 것을 말하고 Lemmataiztion은 표제어 추출로 단어들을 기본 사전형 단어인 표제어로 변환하는 과정임.

Bag-of-Words : 가장 단순한 벡터화 방법 중 하나로 문서나 문법에서 단어의 순서는 무시하고 단순히 단어의 빈도만을 고려해 벡터화하는 과정

TF-IDF : 단어를 벡터화를 진행할 때 다른 문서에 등장하지 않는 단어에 가중치를 두어 특정 문서에만 등장하는 단어의 대표 단어를 선정하는 방식


**N322**
- Word2Vec
- fastText

***이곳에 답안을 입력해주세요***

Word2Vec : 말 그대로 단어를 벡터로 나타내는 방법으로 쉽게 사용되는 임베딩 방법 중에 하나

fastText : 단어를 벡터로 만드는 기법 중 하나로 Word2Vec과 가장 큰 차이점은 Word2Vec은 단어를 쪼개질 수 없는 단위라고 생각하지만 fastText는 하나의 단어에도 여러개의 단어가 존재하는 것으로 인식함.

**N323**
- RNN
- LSTM, GRU
- Attention

***이곳에 답안을 입력해주세요***

RNN : 순환 신경망으로 출력되는 벡터가 다시 입력되는 특성이 있기 때문에 순환 신경망이라는 이름이 붙음. 장점으로는 어떤 길이의 sequential 데이터라도 처리할 수 있고, 단점으로는 병렬화 불가, 기울기 소실 등이 있음.

LSTM, GRU : LSTM은 RNN의 구조에서 기울기 소실 문제를 해결하기 위해 forget, input, output 게이트를 추가한 것으로 뒷쪽 시퀀스의 정보에 비중을 결정함과 동시에 앞쪽 시퀀스 정보를 잃지 않는 방법임, GRU는 LSTM의 구조를 조금 더 단순화 한 것으로 하나의 게이트에서 forget, input 게이트를 제어하고 LSTM구조에 있던 cell-state를 통일시켜 제거하고 output 게이트도 제거한 것

Attention : RNN의 단점인 기울기 소실로부터의 장기 의존성 문제를 해결하기 위해, 각 인코더의 Time-step마다 생성되는 hidden-state 벡터를 간직하는 방식

# Part 2 : Fake/Real News Dataset

한 주간 자연어처리 기법을 배우면서 여러분은 다양한 기술들을 접했습니다.<br/>
어떻게 텍스트 데이터를 다뤄야 하는지, 텍스트를 벡터화 하는 법, 문서에서 토픽을 모델하는 법 등 다양한 NLP 기법을 배웠는데요.<br/>
이번 스프린트 챌린지에선 [Fake/Real News Dataset](https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset)을 사용하여 배운 것들을 복습해보는 시간을 갖겠습니다.

**주의 : 모델의 성능을 최대한 끌어올리는 것이 아닌 모델 구동에 초점을 맞춰주세요.<br/>
모든 문제를 완료한 후에도 "시간이 남았다면" 정확도를 올리는 것에 도전하시는 것을 추천드립니다.**

In [36]:
# 코드 실행 전 seed를 지정하겠습니다.
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
%matplotlib inline

np.random.seed(42)
tf.random.set_seed(42)

## 2.0 데이터셋을 불러옵니다.

- 위 캐글 링크에서 데이터셋을 받아 업로드 합니다.<br/>
(직접 업로드하게 되면 시간이 꽤 걸리므로 **drive_mount** 나 **kaggle 연동**하시는 것을 추천드립니다.)

- 'label' 열을 만들어 Fake = 1, True = 0 로 레이블링해줍니다.
- 두 파일을 합쳐 하나의 데이터프레임에 저장해 준 후 데이터를 섞어줍니다.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df1 = pd.read_csv('/content/drive/MyDrive/colab/Fake.csv')
df2 = pd.read_csv('/content/drive/MyDrive/colab/True.csv')

In [7]:
df1['label'] = 1
df2['label'] = 0

In [10]:
df = pd.concat([df1,df2])
df_total = df.sample(frac=1).reset_index(drop=True)

In [11]:
df_total.head()

,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",1
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",0
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",0
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",1
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",0


In [12]:
df = df_total

## 2.1 TF-IDF 를 활용하여 특정 뉴스와 유사한 뉴스 검색하기

시간상 특별한 **전처리 없이** 아래 태스크를 수행하겠습니다.

### 2.1.1 TFidfVectorizer를 사용하여 문서-단어 행렬(Document-Term Matrix) 만들기

In [13]:
# 이 곳에 답안을 작성하시길 바랍니다.
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

tfidf = TfidfVectorizer(stop_words='english', max_features=3000)
dtm_tfidf = tfidf.fit_transform(df['text'])
dtm_tfidf = pd.DataFrame(dtm_tfidf.todense(), columns=tfidf.get_feature_names())
dtm_tfidf

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,00,000,10,100,11,12,13,14,15,16,...,yes,yesterday,york,young,youth,youtube,zero,zika,zone,zuma
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.031768,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.029483,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44893,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
44894,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.053852,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
44895,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
44896,0.0,0.0,0.0,0.0,0.0,0.0,0.462668,0.130156,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.030496,0.0


### 2.1.2 KNN 알고리즘을 사용하여 유사한 문서 검색하기

- **42번 인덱스의 문서**와 가장 유사한 **5개 문서(42번 포함)의 인덱스**와 **해당 인덱스의 레이블**을 나타내주세요.
- NN 모델의 파라미터 중 `algorithm = 'kd_tree'` 로 설정합니다.

In [14]:
# 이 곳에 답안을 작성하시길 바랍니다.
nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm_tfidf)

NearestNeighbors(algorithm='kd_tree')

In [15]:
nn.kneighbors([dtm_tfidf.iloc[42]])

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


(array([[0.        , 0.        , 0.98598258, 0.98690012, 1.        ]]),
 array([[   42, 33954, 24997, 15323,  1639]]))

답 : [42, 33954, 24997, 15323, 1639]

## 2.2 Keras Embedding을 사용하여 분류하기

### 2.2.0 데이터셋 split

- Train, Test 데이터셋으로 분리(Split)하여 주세요.

In [49]:
# 이 곳에 답안을 작성하시길 바랍니다.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2)

In [50]:
X_train.shape, y_train.shape

((35918,), (35918,))

### 2.2.1 단어 벡터의 평균을 이용하여 분류해보기

N322에서 했던 단어 임베딩 벡터의 평균을 사용하여 문장을 분류하는 작업을 수행해봅시다.<br/>
인스턴스마다 텍스트 길이가 길고 시간이 오래 걸리므로 시간상 epoch 수를 **10 이하**로 하는 것을 추천드립니다.<br/>
모델 구동이 목적이므로 임베딩 차원 수를 크지 않게(50이하)로 설정해주세요.<br/>
**권장사항 : `max_len` 은 텍스트 길이 평균보다 높게 설정해주세요.**<br/>

> **Tip : 모델이 학습하는 동안 2.2.3의 내용을 작성하면 시간을 절약할 수 있습니다.**


In [52]:
X_train.str.len().mean()

2465.175009744418

In [53]:
# 이 곳에 답안을 작성하시길 바랍니다
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer

sentences = [txt for txt in X_train]

tokenizer = Tokenizer(num_words = 50)
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1
X_train_encoded = tokenizer.texts_to_sequences(sentences)

max_len = 2500
X_train=pad_sequences(X_train_encoded, maxlen=max_len, padding='post')

test_sentences = [txt for txt in X_test]
X_test_encoded = tokenizer.texts_to_sequences(test_sentences)
X_test=pad_sequences(X_test_encoded, maxlen=max_len, padding='post')

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300))
def get_vector(word):
    """
    입력 단어가 vocab 에 있는 단어일 경우 임베딩 벡터를 반환
    
    Args:
        word: 입력 단어 -> str
    """
    if word in df:
        return df[word]
    else:
        return None

for word, i in tokenizer.word_index.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp

In [60]:
model1 = Sequential()
model1.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False))
model1.add(GlobalAveragePooling1D()) # 입력되는 단어 벡터의 평균을 구하는 함수입니다.
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model1.fit(X_train, y_train, batch_size=64, epochs=10, validation_split=0.2)  

Epoch 1/10
449/449 [==============================] - 90s 199ms/step - loss: 0.6924 - acc: 0.5239 - val_loss: 0.6927 - val_acc: 0.5146
Epoch 2/10
449/449 [==============================] - 95s 211ms/step - loss: 0.6920 - acc: 0.5246 - val_loss: 0.6928 - val_acc: 0.5146
Epoch 3/10
449/449 [==============================] - 101s 226ms/step - loss: 0.6920 - acc: 0.5246 - val_loss: 0.6929 - val_acc: 0.5146
Epoch 4/10
449/449 [==============================] - 84s 188ms/step - loss: 0.6920 - acc: 0.5246 - val_loss: 0.6929 - val_acc: 0.5146
Epoch 5/10
449/449 [==============================] - 84s 187ms/step - loss: 0.6919 - acc: 0.5246 - val_loss: 0.6928 - val_acc: 0.5146
Epoch 6/10
449/449 [==============================] - 89s 199ms/step - loss: 0.6920 - acc: 0.5246 - val_loss: 0.6929 - val_acc: 0.5146
Epoch 7/10
449/449 [==============================] - 82s 183ms/step - loss: 0.6920 - acc: 0.5246 - val_loss: 0.6929 - val_acc: 0.5146
Epoch 8/10
449/449 [==============================] - 

In [61]:
result1 =  np.round(model1.evaluate(X_test, y_test),4)

281/281 [==============================] - 24s 84ms/step - loss: 0.6919 - acc: 0.5245


### 2.2.2 LSTM을 사용하여 텍스트 분류 수행해보기

N323에서 했던 단어 임베딩 벡터의 평균을 사용하여 문장을 분류하는 작업을 수행해봅시다.<br/>
인스턴스마다 텍스트 길이가 길어 시간이 매우 오래 걸리므로 <br/>
**층을 최소한으로 쌓고**, epoch 수를 **3 이하**로 하는 것을 추천드립니다.<br/>

> **Tip : 모델이 학습하는 동안 2.2.3의 내용을 작성하면 시간을 절약할 수 있습니다.**


In [33]:
from keras.callbacks import EarlyStopping

In [54]:
# 이 곳에 답안을 작성하시길 바랍니다
max_words = 50
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [55]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 2500)]            0         
                                                                 
 embedding_1 (Embedding)     (None, 2500, 50)          2500      
                                                                 
 lstm_1 (LSTM)               (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_2 (Activation)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257 

In [56]:
model.fit(X_train, y_train, batch_size=128, epochs=3,
          validation_split=0.2)

Epoch 1/3
225/225 [==============================] - 1670s 7s/step - loss: 0.6924 - accuracy: 0.5227 - val_loss: 0.6936 - val_accuracy: 0.5146
Epoch 2/3
225/225 [==============================] - 1608s 7s/step - loss: 0.6923 - accuracy: 0.5230 - val_loss: 0.6929 - val_accuracy: 0.5146
Epoch 3/3
225/225 [==============================] - 1571s 7s/step - loss: 0.6921 - accuracy: 0.5244 - val_loss: 0.6932 - val_accuracy: 0.5146


In [58]:
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss, Accuracy =  ({:0.4f}, {:0.4f})'.format(accr[0],accr[1]))

281/281 [==============================] - 109s 388ms/step - loss: 0.6920 - accuracy: 0.5245
Test set
  Loss, Accuracy =  (0.6920, 0.5245)


### 2.2.3 위에서 실행한 내용에 대해 다시 알아봅시다.

#### a) 데이터셋을 학습할 때 사용하는 `pad_sequences`  메서드에 대해 설명해주세요.<br/>어떤 기능을 하나요? 모델을 학습할 때 왜 필요한가요?

***이곳에 답안을 입력해주세요***

입력되는 시퀀스의 길이가 모두 다르기 때문에 max_len을 통해 입력되는 단어의 길이를 맞춰주기 위해 입력의 길이의 뒤나 앞에 0을 넣어주는 작업을 말함. 모델을 학습할 때 이 작업이 필요한 이유는 신경망에 대한 입력은 길이가 같아야 하므로 이 과정을 진행해야 함.

#### b) 2.2.1과 2.2.2에서 사용한 각 모델의 evaluation 성능은 어떻게 나왔나요?<br/>각 모델의 장단점은 무엇이라고 생각하나요?

***이곳에 답안을 입력해주세요***

#### c) 종래의 RNN(Recurrent Neural Networks) 대신 LSTM(Long-Short Term Memory)을 사용하는 이유는 무엇인가요?<br/>(i.e. RNN에 비해 LSTM의 좋은 점을 설명해주세요.)

***이곳에 답안을 입력해주세요***

LSTM은 RNN의 구조에서 기울기 소실 문제를 해결하기 위해 forget, input, output 게이트를 추가한 것으로 뒷쪽 시퀀스의 정보에 비중을 결정함과 동시에 앞쪽 시퀀스 정보를 잃지 않는 않기 때문에 더 좋음. 따라서 RNN에서 일어나는 역전파시 기울기 소실 문제를 추가되는 게이트를 통해 기울기 소실 문제를 해결하는 방법임.

#### d) LSTM이나 RNN을 사용하는 예시를 **3개**이상 제시하고 해당되는 경우에 왜 LSTM이나 RNN을 사용하는 것 적절한지 간단하게 설명해주세요.

***이곳에 답안을 입력해주세요***

1. 주가예측 : LSTM이 더 좋음. RNN은 재귀적이기 때문에 손실함수의 일차 미분값은 계속해서 감소하기 때문에 결국 소실하기 때문에 주가를 예측하는 과정에서 오차를 측정하는 손실함수가 사라지면 주가 예측의 오차가 매우 커질 수 있기 때문.(장기 메모리를 기억하는 것이 좋음)

2. 이메일 스팸 탐지기 : RNN을 사용해도 무관함. 출력과 먼 곳에 있는 데이터를 무시하더라도 스팸메일의 경우 점점 최근의 방식으로 적용되며 변화하기 때문에 장기 메모리는 무시해도 될 것 같음.

3. 챗봇 고객 서비스 : LSTM이 더 좋음. 과거의 데이터를 소실하여 출력하게 된다면 오차가 더 커짐.

#### e) 이외에 N324 에서 배운 자연어처리 모델과 관련된 키워드를 3개 이상 적어주세요. <br/> (해당 키워드에 대한 설명은 옵션입니다.)

***이곳에 답안을 입력해주세요***
- GPT : Generative Pre-trained Transformer 의 약자로 사전 학습과 관련된 모델
- Fine-tuning : 사전 학습이 끝난 모델에 우리가 하고자하는 태스크에 특화된 데이터를 학습하는 과정
- BERT : Bidirectional Encoder Representation by Transformer의 약자로 트랜스포머의 인코더만을 사용하여 문맥을 양방향으로 읽어내는 과정.
- MLM : BERT 방식의 사전학습에 사용되는 방법
- NSP : BERT 방식의 사전학습에 사용되는 방법으로 문맥에 맞는지 아닌지를 판단하여 학습하는 방식

# Advanced Goals: 3점을 획득하기 위해선 아래의 조건 중 하나 이상을 만족해야합니다
 
- 2.1 에서 TF-IDF(`TfidfVectorizer`)가 아닌 방법을 사용하여 유사도 검색을 수행해보세요.<br/>
TF-IDF와 해당 방법의 차이를 설명해주세요. 
- 2.2 에서 사용한 방법을 재사용하되 하이퍼 파라미터를 조정하거나 모델 구조를 변경하여 성능을 올려봅시다.<br/>**(주의 : GridSearch, RandomSearch 등의 방법을 사용하여도 좋으나 시간이 오래 걸리므로 범위를 잘 선택해야 합니다.)**

In [62]:
# 이 곳에 답안을 작성하시길 바랍니다
# 2.1 
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words='english', max_features=3000)

dtm_count = count_vect.fit_transform(df['text'])

dtm_count = pd.DataFrame(dtm_count.todense(), columns=count_vect.get_feature_names())
dtm_count
nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm_count)
nn.kneighbors([dtm_tfidf.iloc[42]])

# 차이는 TF-IDF 방식은 가중치를 활용하여 모든 단어에 가중을 두는 것이 아닌
# 다른 문장에서 나오지 않는 단어에 가중치를 많이 줌으로써 많이 나오는 단어의 가중치를 줄임.

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


(array([[1., 1., 1., 1., 1.]]), array([[424,  11, 177, 400, 324]]))

In [82]:
# 2.2
def model_builder():
  model = Sequential()
  model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(GlobalAveragePooling1D()) # 입력되는 단어 벡터의 평균을 구하는 함수입니다.
  model.add(Dense(1, activation='sigmoid')) # 이진분류니까 노드수 1, 활성함수로는 시그모이드
  model.compile(optimizer='adam', 
                loss='binary_crossentropy', 
                metrics=['accuracy'])

  return model

In [83]:
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [84]:
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

In [ ]:
# keras.wrapper를 활용하여 분류기를 만듭니다
model = KerasClassifier(build_fn = model_builder)

# GridSearch
parameters = {'batch_size': [16, 32],
              'epochs': [3],
              'optimizer': ['adam', 'rmsprop']}

grid = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 3)
grid_result = grid.fit(X_train, y_train, validation_data=(X_test, y_test))

/usr/local/lib/python3.8/dist-packages/scikeras/wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


Epoch 1/3
1497/1497 [==============================] - 68s 45ms/step - loss: 0.6923 - accuracy: 0.5209 - val_loss: 0.6920 - val_accuracy: 0.5245
Epoch 2/3
1497/1497 [==============================] - 66s 44ms/step - loss: 0.6922 - accuracy: 0.5226 - val_loss: 0.6920 - val_accuracy: 0.5245
Epoch 3/3
749/749 [==============================] - 22s 29ms/step


/usr/local/lib/python3.8/dist-packages/scikeras/wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


Epoch 1/3
1497/1497 [==============================] - 63s 42ms/step - loss: 0.6923 - accuracy: 0.5216 - val_loss: 0.6920 - val_accuracy: 0.5245
Epoch 2/3
1497/1497 [==============================] - 68s 45ms/step - loss: 0.6922 - accuracy: 0.5226 - val_loss: 0.6920 - val_accuracy: 0.5245
Epoch 3/3
749/749 [==============================] - 23s 31ms/step


/usr/local/lib/python3.8/dist-packages/scikeras/wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


Epoch 1/3
1497/1497 [==============================] - 65s 42ms/step - loss: 0.6922 - accuracy: 0.5222 - val_loss: 0.6920 - val_accuracy: 0.5245
Epoch 2/3
1497/1497 [==============================] - 64s 43ms/step - loss: 0.6921 - accuracy: 0.5226 - val_loss: 0.6920 - val_accuracy: 0.5245
Epoch 3/3
749/749 [==============================] - 23s 30ms/step


/usr/local/lib/python3.8/dist-packages/scikeras/wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


Epoch 1/3
1224/1497 [=======================>......] - ETA: 8s - loss: 0.6924 - accuracy: 0.5207

In [ ]:
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")